In [15]:
# Import Libraries
# for text preprocessing
import re
# import spacy
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
# import numpy for matrix operation
import numpy as np
# Importing Gensim
import gensim
from gensim import corpora  
import pandas as pd

In [16]:
import ast
dataframe = pd.read_csv("./data/preprocessed.csv")
# clean_corpus = dataframe["preprocessed"]
# clean_corpus = [x.strip('][').split(', ') for x in dataframe["preprocessed"]]
# string to list
clean_corpus = [ast.literal_eval(x) for x in dataframe["preprocessed"]]

In [17]:
# where every unique term is assigned an index.
dict_ = corpora.Dictionary(clean_corpus)

In [18]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary
doc_term_matrix = [dict_.doc2bow(i) for i in clean_corpus]

In [19]:
# Uncomment to retrain LDAModel
%%script false
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Training LDA model on the document term matrix.
ldamodel = Lda(corpus=doc_term_matrix, num_topics=6, id2word = dict_, passes=20, random_state=20, eval_every=None)

# Prints the topics with the indexes: 0,1,2 :
ldamodel.print_topics()
# we need to manually check whethere the topics are different from one another or not
print(ldamodel.print_topics(num_topics=6, num_words=5))

# num_topics mean: how many topics want to extract
# num_words: the number of words that want per topic


UsageError: Line magic function `%%script` not found.


In [20]:
# Comment to prevent loading the pretrained model
# %%script false

import pickle

ldamodel = None
with open('./out/lda_model.pkl', 'rb') as f:
    ldamodel = pickle.load(f)

In [21]:
from gensim.models import CoherenceModel

coherence_model = CoherenceModel(model=ldamodel, texts=clean_corpus, dictionary=dict_, coherence="c_v")

coherence_model.get_coherence() 

0.6086242575183766

In [22]:
# NOTE: Gensim’s perplexity value is in logarithmic form. To compare with sklearn’s perplexity value np.exp(-1 *gensim.log_perplexity) is used

print(ldamodel.log_perplexity(doc_term_matrix ))
print(np.exp(-1 * ldamodel.log_perplexity(doc_term_matrix )))


-11.11546611000127
67203.01403915934


In [23]:
import numpy as np
all_rows = []

for index,doc_in_words in enumerate(clean_corpus):
    doc_in_words_as_string = ' '.join(doc_in_words)
    print(doc_in_words_as_string)
    row_vals = [dataframe.loc[index,'platform'], doc_in_words_as_string]+list(np.zeros(6))
    doc_topics = ldamodel.get_document_topics(dict_.doc2bow(doc_in_words))
    for doc_topic in doc_topics:
        row_vals[doc_topic[0]+2] = doc_topic[1]
    index_of_best_topic = np.argmax(row_vals[2:])
    row_vals.append(index_of_best_topic+1)
    if row_vals[1] == False or row_vals[1] == "":
        print(row_vals)
    all_rows.append(row_vals)


nowadays every thing seem increasing governance control greater good humanity selfish desire power wealth lord bless country
disagree high
wag tayong magalala naniniwala isusuprise sir bbm tallano gold pasko tuloy tuloy unity party party woohoo
ok yang nang bansa ganyan selfish fanatic blengblong
samasama tayong babaon
kasing bilis formula race lol
nagkakaisa bilihin magtaas magtaasan tunay unity
shout bumoto deserve guy really
think number high
whole world affected inflation
dont worry monitored monitor monitor unity
inflation accelerates
disagree number high unity
maliit bagay nakuha daw business dealing panunuod f racing
nu kya mngyyare bnsa presidente ntin teka teka ksi khit saang bnsa tumaas lht bilihin need tlga bnsa aay mtalino kyang mkiharaap khit saang bnsaa importante proud
punchline bagay high jay ar
mabilis accellerate kesa kotse singapore f grand prix
wag magulo f pinsan fav sonyour future president
tikman golden age
worry inflation important thing unity
balakayojan basta 

In [24]:
import pandas as pd
labelled_dataset = pd.DataFrame(all_rows, columns=["Platform","Text"]+[f"Topic {i+1}" for i in range(6)]+["Best Topic"])
labelled_dataset

,Platform,Text,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Best Topic
0,Facebook,nowadays every thing seem increasing governanc...,0.953512,0.000000,0.000000,0.000000,0.000000,0.000000,1
1,Facebook,disagree high,0.055609,0.055623,0.055557,0.722094,0.055557,0.055561,4
2,Facebook,wag tayong magalala naniniwala isusuprise sir ...,0.000000,0.000000,0.000000,0.010047,0.000000,0.950687,6
3,Facebook,ok yang nang bansa ganyan selfish fanatic blen...,0.134109,0.018535,0.464619,0.018795,0.018535,0.345407,3
4,Facebook,samasama tayong babaon,0.041726,0.041726,0.041726,0.041727,0.285804,0.547292,6
...,...,...,...,...,...,...,...,...,...
4720,Youtube,mukhang nakashabu,0.056021,0.056029,0.384557,0.056027,0.391338,0.056028,5
4721,Youtube,bbm mixed confusing economic term,0.526034,0.027956,0.027957,0.361964,0.027892,0.028196,1
4722,Youtube,mrutal utal,0.055656,0.055656,0.055657,0.388541,0.055657,0.388833,6
4723,Youtube,leni lutang,0.055800,0.055738,0.721113,0.056101,0.055670,0.055578,3


In [25]:
labelled_dataset.to_csv("./data/labelled_dataset.csv",index=False)

In [ ]:
# Dump model
# pickle.dump(ldamodel, open('./out/lda_model.pkl','wb'))